# Simulating the ML model serving 

This notebook is used to simulate the serving of the ML model with an interactive input. 

# Python packages 

In [1]:
# Json reading 
import json 

# Pickle reading 
import pickle

# Operating system functionality 
import os 

# Input simulation 
from ipywidgets import interactive, widgets, interact
from IPython.display import display

# Data wrangling 
import pandas as pd 

# Reading the necesary objects 

In [2]:
# Saving the path to the ML folder 
_ml_folder = os.path.join("..", 'ml_models')
_ml_model_path = os.path.join(_ml_folder, "ml-model-lr.pkl")
_ml_features_path = os.path.join(_ml_folder, "ml-features.json")

# Reading the model object 
model = pickle.load(open(_ml_model_path, 'rb'))
features = json.load(open(_ml_features_path, 'rb'))

# Printing out the features 
print(features)

{'bomb_planted': 'int64', 'ct_health_share': 'float64', 'ct_players_alive': 'float64', 't_players_alive': 'float64', 'ct_defuse_kit_present': 'int64', 'ct_helmets': 'float64', 't_helmets': 'float64'}


In [4]:
# Defining the input preparation function 
def prepare_input(raw_input_dict: dict, features: dict) -> pd.DataFrame:
    """
    Function that accepts the raw input dictionary and the features dictionary and returns a pandas dataframe with the input prepared for the model.
    """
    # Extracting the key names 
    feature_names = list(raw_input_dict.keys())
    original_feature_names = list(features.keys())

    # Ensuring that all the keys present in **features** are in **raw_input_dict**
    missing_features = set(original_feature_names) - set(feature_names)
    if len(missing_features): 
        return print(f"Missing features in input: {missing_features}")

    # Iterating and preprocesing 
    prepared_features = {}
    for feature in feature_names:
        # Extracting the type of the feature 
        feature_type = features.get(feature) 

        # Converting to that type 
        feature_value = raw_input_dict.get(feature)
        
        if feature_type == "float64":
            feature_value = float(feature_value) 
        
        if feature_type == "int64":
            feature_value = int(feature_value)

        # Saving to the prepared features dictionary
        prepared_features[feature] = feature_value 
    
    # Creating a dataframe from the prepared features 
    df = pd.DataFrame(prepared_features, index=[0])

    # Ensuring that the names are in the exact order 
    df = df[original_feature_names]

    # Returning the dataframe 
    return df 


# Interactive serving 

## Defining the input widgets

In [8]:
# Boolean for bomb planting event 
bomb_planted_widget = widgets.Checkbox(
    value=False,
    description='Has the bomb been planted?',
    disabled=False
)

# Boolean for the presence of the difusal kit
ct_defuse_kit_present_widget = widgets.Checkbox(
    value=False,
    description='Is there a difusal kit present in CT team?',
    disabled=False
)

# CT health share of total; the range is (0, 1.0)
ct_health_share_widget = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='CT health share of total')

# Count of CT and T players which are alive
ct_players_alive_widget = widgets.Dropdown(
    options=list(range(0, 6, 1)),
    value=3,
    description='The number of alive CT players',
    disabled=False,
)

t_players_alive_widget = widgets.Dropdown(
    options=list(range(0, 6, 1)),
    value=3,
    description='The number of alive T players',
    disabled=False,
)

# Number of helmets in a team 
ct_helmets_widget = widgets.Dropdown(
    options=list(range(0, 6, 1)),
    value=3,
    description='CT helmets',
    disabled=False,
)

t_helmets_widget = widgets.Dropdown(
    options=list(range(0, 6, 1)),
    value=3,
    description='T helmets',
    disabled=False,
)


## Simulation function

In [9]:
def get_prob(
    bomb_planted, 
    ct_defuse_kit_present,
    ct_health_share,
    ct_players_alive,
    t_players_alive,
    ct_helmets, 
    t_helmets
    ):
    """
    Interactive session to experiment with the created ML model 
    """
    # Creating the raw input dictionary 
    raw_input = {
        "bomb_planted": bomb_planted,
        "ct_defuse_kit_present": ct_defuse_kit_present,
        "ct_health_share": ct_health_share, 
        "ct_players_alive": ct_players_alive,
        "t_players_alive": t_players_alive,
        "ct_helmets": ct_helmets,
        "t_helmets": t_helmets
    }

    # Preparing the input for the model feature importance plot xgboost
    raw_input_df = prepare_input(raw_input, features)

    # Getting the probabilities 
    p = model.predict_proba(raw_input_df)[0]
    p_win = round(p[1], 3)

    # Returning the probabilities 
    print(f"Probability of CT winning: {p_win}")

## Simulation application 

In [10]:
# Making the interactive session 
prob_widget = interactive(
    get_prob, 
    bomb_planted=bomb_planted_widget, 
    ct_defuse_kit_present=ct_defuse_kit_present_widget,
    ct_health_share=ct_health_share_widget,
    ct_players_alive=ct_players_alive_widget,
    t_players_alive=t_players_alive_widget,
    ct_helmets=ct_helmets_widget,
    t_helmets=t_helmets_widget,
)

# Displaying the widget 
prob_widget

interactive(children=(Checkbox(value=False, description='Has the bomb been planted?'), Checkbox(value=False, d…